## Problem Set 3

Please work on Python coding exercises. You should add a new cell right below each exercise to write down your code and execute the code. **This assignment is due on Jan 19th.**

## Exercise 1

In the class, we numerically solved a neoclassical growth model using the *policy function iteration* algorithm. However, the actual code was not following the PFI algorithm specified in the lecture slides.

In this exercise, you will solve the neoclassical growth model using a (modfied version of) policy function iteration algorithm written in the lecture slides. More specifically, you write a code based on the algorithm below. **Please check if the policy function converges to a true policy function.**

## Policy Function Iteration Algorithm

An alternative way of solving the problem is a *policy function iteration* or *Howard's improvement algorithm*. As its name indicates, this algorithm starts with a guess on the policy function and keeps updating the policy function instead of value function. Sometimes, this algorithm solves the problem a little faster than the VFI. Here is a brief sketch of the algorithm.

In the neoclassical growth model setting, let $h()$ be the policy function, $k' = h(k)$.

1. Choose any $ h^0 \in \mathbb{R}^n $, and specify $ \varepsilon > 0 $; set $ i = 0 $  
1. From the initial guess of the policy function, for each $k$ we have $\tilde{k}' = h^0(k)$.
1. Given $(\tilde{k}',k)$, we can compute $u(c) = u(f(k)+(1-\delta) - \tilde{k}')$.
1. If our policy function $h^i$ for any iteration $i=0,1,2,...$ was the optimal specification $h^*$, then the max operator in the bellman equation should be eliminated:
\begin{equation}
v^* = u(f(k)+(1-\delta)k-h^*) + \beta v^*
\end{equation}
Then, from this specification, we can solve for $v^*$: 
\begin{equation}
v^* = \frac{1}{1-\beta}u(f(k)+(1-\delta)k-h^*)
\end{equation}
If $h^i$ is not optimal, then the equation above serves as an updating equation. Therefore, by the $i$-th iteration, the updated value function becomes
\begin{equation}
v^{i+1} = \frac{1}{1-\beta}u(f(k)+(1-\delta)k-h^{i})
\end{equation}
1. Using the updated value function, obtain a new policy function for the next iteration:
\begin{equation}
h^{i+1} = \underset{k'}{\operatorname{argmax}} u(f(k)+(1-\delta)k-k') + \beta v^{i+1}
\end{equation}
1. If $ \lVert h^{i+1} - h^i\rVert <  [(1 - \beta) / (2\beta)] \varepsilon $,
  then stop; otherwise, set $ i = i + 1 $ and go to step 4  


In [6]:
import numpy as np

class ngm_pfi:
    # INITIALIZATION OF NGM CLASS WITH GIVEN PREDETERMINED PARAMETERS VALUES
    def __init__(self,  nk      = 1000,
                        α       = 0.34,
                        β       = 0.95,
                        σ       = 2,
                        δ       = 0.04):

        # GRID FOR k
        self.nk             = nk

        # MODEL PARAMETERS
        self.α              = α
        self.β              = β
        self.σ              = σ
        self.δ              = δ
        
        # COMPUTE STEADY STATE CAPITAL-LABOR RATIO
        self.k_ss           = ((1/β-1+δ)/α)**(1/(α-1))

        # INITIALIZE GRID POINTS
        self.kgrid          = self.init_kgrid()
        
        # INITIALIZE VALUE FUNCTION AND POLICY FUNCTION
        self.kp             = self.kgrid
        cons                = self.kgrid**self.α + (1 - self.δ)*self.kgrid - self.kp
        self.V              = self.util(cons)/(1-self.β)


    def init_kgrid(self):
        '''
        This function constructs and returns grid points for state k.
        The grid points are in equidistance.
        '''
        # THE MIN AND MAX VALUES OF k RANGE
        kmin = 0.5*self.k_ss
        kmax = 1.5*self.k_ss

        # DISCRETIZE THE RANGE WITH NK GRID POINTS BY AN EQUIDISTANCE
        self.kgrid = np.linspace(kmin,kmax,self.nk)
        
        # RETURN THE GRID POINTS
        return self.kgrid


    def util(self,cons):
        '''
        This function returns the value of CRRA utility with σ
        u(c) = c**(1-σ)/(1-σ)
        '''
        if self.σ != 1:
            uu = cons**(1-self.σ)/(1-self.σ)
        else:
            uu = np.log(cons)
        return uu

    def get_V(self):
        '''
        This function returns the value function
        '''
        return self.V

    def get_kp(self):
        '''
        This function returns the policy function
        '''
        return self.kp
    
    def set_V(self,VV):
        '''
        This function updates the value function
        '''
        self.V   = VV
        
    def set_kp(self,kp):
        '''
        This function updates the policy function
        '''
        self.kp   = kp        

In [7]:
def bellman(kp,k0,kgrid,VV,α,β,σ,δ):
    '''
    This function computes bellman equation for a given state k0.
    Input:
        kp: an evaluating point of k'
        k0: current state of capital-labor ratio
        kgrid: predetermined discrete state space
        VV: next period's value function at k'
        α,β,σ,δ: model parameters
    Output:
        -vv: value function (negative so that fminbound search for a minimum value)
    ''' 

    # Interpolate next period's value function evaluated at k'
    # using 1-dimensional interpolation function in numpy
    V1         = np.interp(kp,kgrid,VV)
        
    # Interpolated value cannot be NaN or Inf
    if np.isnan(V1) or np.isinf(V1): print("bellman: V1 is NaN.")

    # Compute consumption at a given k0 and k'
    cons = k0**α + (1 - δ)*k0 - kp
    
    # Consumption must be non-negative
    if cons<=0:
        # Assign some large negative values
        vv = cons*1000
    else:
        # Compute value function
        vv  = cons**(1-σ)/(1-σ) + β*V1
    
    return -vv

 (a) Please complete the missing code (...) below.

In [ ]:
from scipy.optimize import fminbound
import sys
import matplotlib.pyplot as plt
import time


# Set values on model parameters
α, β, σ, δ, nk = 0.34, 0.95, 2, 0.01, 100

# Instantiate NGM object
hh = ngm_pfi(nk, α, β, σ, δ)

# Create grid points over the state space
kgrid = hh.init_kgrid()

# Set tolerance parameter and an arbitrary initial difference
tol = 1e-5
diff = 1e5
niter  = 0
    
#Iterate until the difference of old and new value functions are less than the tolerance level
tic = time.perf_counter()
while diff>tol and niter<500:

    # Start with the current policy function
    kp_old     = ...  # policy function from the previous iteration: h(k(t)) = k(t+1)
    cons_old   = ...  # compute consumption associated with the policy function
    V_old      = ...  # compute value function from the updating rule
    kp_new     = np.zeros(nk)   # empty policy function
    
    # Solve for optimal capital and value function at each point of state k
    for ik in range(nk):
        k0 = kgrid[ik]
        kmin = kgrid[0]
        kmax = kgrid[nk-1]
        kp_new[ik] = fminbound(bellman,kmin,kmax,args=(k0,kgrid,V_old,α,β,σ,δ))
        
    # Update the policy functions
    hh.set_kp(kp_new)
    
    # Compute the difference between new and old policy functions by sup norm
    diff = np.max(abs(...))
    niter += 1
    if niter%10==0:
        sys.stdout.write('Number of Iteration = %d, Difference = %f\n' % (niter,diff))

toc = time.perf_counter()
sys.stdout.write('Value function converged. The elapsed time = %f seconds.\n' % (toc-tic))

# Plot Value Function
fig, ax = plt.subplots(figsize=(7, 5))
plt.plot(hh.kgrid, hh.V,label="$V(k)$")
plt.legend(loc='lower right', fontsize = 14)
plt.show()

# Plot Policy Function
fig, ax = plt.subplots(figsize=(7, 5))
plt.plot(hh.kgrid, hh.kp,label="$k'=g(k)$")
plt.plot(hh.kgrid, hh.kgrid,label="$k=k'$")
plt.legend(loc='lower right', fontsize = 14)
plt.show()

(b) Can you find out as to why this algorithm doesn't work to get a convergence on the policy function?